In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
roi = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan').first()).geometry()
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,8)

In [4]:
# 添加一个底图白板
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
Map.addLayer(basemap,{'palette':['white']},"basemap")

In [5]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

visParams = {
    'bands': ['B5', 'B6', 'B3'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

def ndvi_mndwi(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    class_ndvi_mndwi = ndvi_mndwi.lt(0).rename('class_ndvi_mndwi')
    class_mndwi = mndwi.gt(0).rename('class_mndwi')
    return image.addBands(ndvi_mndwi).addBands(ndvi).addBands(mndwi).addBands(class_ndvi_mndwi).addBands(class_mndwi)

# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def AWEIsh(image):
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(awei)

# 计算单景影像上的分类结果

## 获取land、seasonal-water、permanent-water的范围

In [6]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(ndvi_mndwi)

# ndvi_mndwi水体频率
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
water_layer = landsat_images.map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
validPixel_layer = landsat_images.count().select('B3').clip(roi).rename('count')
index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()
# JRC
JRC_2018 = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC_2018,basemap]).sum()
# Maryland
Maryland_2018 = ee.Image('users/311605001111/Maryland_2018').clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland_2018.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= ee.ImageCollection([Maryland_2018.gte(25),Maryland_2018.lt(75)]).sum().eq(2)
Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# 三者的交集
permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(3)
landExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(3)
seasonWaterExtent = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')

## 在单景影像上做验证

In [7]:
image9 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_123039_20180915')
awei = AWEIsh(image9)
Map.addLayer(awei.clip(roi),visParams,'image9')


In [8]:
permanent = permanentWaterExtent.clip(image9.geometry()).remap([0,1],[0,2]).rename('waterclass')
# Map.addLayer(permanent.selfMask(),{'palette':['blue'],'min':0,'max':1},'permanentwaterExtent')


season_image = image9.clip(roi).updateMask(seasonWaterExtent)
# Map.addLayer(season_image,visParams,'seasonal')
season_class = AWEIsh(season_image).select('AWEI').gt(0).rename('waterclass')
# Map.addLayer(season_class,{'palette':['#636363','blue'],'min':0,'max':1},'seasonClass')
seasonalwater_Extent = ee.ImageCollection([season_class,basemap]).sum().remap([0,1],[0,2]).rename('waterclass').clip(image9.geometry()) 
# Map.addLayer(seasonalwater_Extent,{'palette':['#636363','blue'],'min':0,'max':1},'seasonalWater')

land = landExtent.clip(image9.geometry())
# Map.addLayer(land.selfMask(),{'palette':['cyan']},'landExtent')


water_clas = ee.ImageCollection([permanent,seasonalwater_Extent,land]).sum().remap([0,1,2],[0,0,1]).rename('train_waterclass')
# Map.addLayer(water_clas,{'palette':['cyan','blue'],'min':0,'max':1},'water_clas')
Map.addLayer(water_clas.selfMask(),{'palette':['blue']},'water_clas')

# 人工选取样本点

在地图上人工选取是“水体”样本点，然后执行后续代码

In [ ]:
points = Map.user_rois
print(points.size().getInfo())

water = ee.FeatureCollection(Map.user_rois).map(lambda i : i.set({'waterclass':1}))

vis2 = {'color': 'blue', 'pointSize': 6, 'pointShape': 'circle', 'width': 2, 'lineType': 'solid', 'fillColor': 'blue'}
Map.addLayer(water,vis2,'points')

将选取的样本点导出

In [ ]:
geemap.ee_to_shp(ee_object=water,filename='samples_water.shp',keep_zip=True)

在地图上人工选取是“陆地”样本点，然后执行后续代码

In [ ]:
points = Map.user_rois
print(points.size().getInfo())

land = ee.FeatureCollection(Map.user_rois).map(lambda i : i.set({'waterclass':0}))

vis1 = {'color': 'ff0d00ff', 'pointSize': 6, 'pointShape': 'circle', 'width': 2, 'lineType': 'solid', 'fillColor': 'f12704a8'}
Map.addLayer(land,vis1,'points')

将选取的样本点导出

In [ ]:
geemap.ee_to_shp(ee_object=land'd,filename='samples_land.shp',keep_zip=True)

# 加载作者已经选取的样本点

In [ ]:
samples_land = ee.FeatureCollection('users/311605001111/samples_land')
samples_water = ee.FeatureCollection('users/311605001111/samples')
samples_all = samples_water.merge(samples_land)

vis1 = {'color': 'ff0d00ff', 'pointSize': 6, 'pointShape': 'circle', 'width': 2, 'lineType': 'solid', 'fillColor': 'f12704a8'}
Map.addLayer(samples_land,vis1,'samples_land')

vis2 = {'color': 'blue', 'pointSize': 6, 'pointShape': 'circle', 'width': 2, 'lineType': 'solid', 'fillColor': 'blue'}
Map.addLayer(samples_water,vis2,'samples')

print(samples_land.size().getInfo())
print(samples_water.size().getInfo())

In [ ]:
#精度验证

# Overlay the points on the imagery to get training.
sample_point = water_clas.sampleRegions(**{
    'collection': samples_all,
    'properties': ['waterclass'],
    'scale': 30
})
print(sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = sample_point.errorMatrix('waterclass', 'train_waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

In [ ]:
## 筛选的11景影像
# image1 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_122039_20180111')
# image2 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_122039_20180212')
# image3 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_123038_20180323')
# image4 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_123038_20180408')
# image6 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_123039_20180627')
# image7 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_123039_20180729')
# image8 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_122039_20180807')
# image9 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_123039_20180915')
# image10 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_123039_20181017')
# image11 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_123039_20181102')
# image12 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_122039_20181213')
# images_col = ee.ImageCollection([image1,image2,image3,image4,image6,image7,image8,image9,image10,image11,image12])
# print(images_col.size().getInfo())
# Map.addLayer(image9.clip(roi),visParams,'image9')